In [1]:
# Import libraries
from pathlib import Path
import pandas as pd
from datetime import datetime
from uszipcode import SearchEngine
import pprint

In [2]:
# Set search engine into a variable
search_geo = SearchEngine()

## Clean sales data

In [3]:
# Import and concat 2019, 2020 data
df_sales_2019 = pd.read_excel(Path("data/weekly_revenue_2019.xlsx"))
df_sales_2020 = pd.read_excel(Path("data/weekly_revenue_2020.xlsx"))
df = pd.concat([df_sales_2019, df_sales_2020], axis='rows', ignore_index=True)
df.head()

,id,fscl_yr_num,wk1_youtube,wk1_tiktok,wk1_ttl,wk2_youtube,wk2_tiktok,wk2_ttl,wk3_youtube,wk3_tiktok,...,wk50_ttl,wk51_youtube,wk51_tiktok,wk51_ttl,wk52_youtube,wk52_tiktok,wk52_ttl,ytd_youtube,ytd_tiktok,ytd_ttl
0,002G,2019,719.154302,521.411670,1168.012197,700.197460,705.117986,1332.761670,773.804302,175.349565,...,902.433249,843.513249,505.312197,1276.271670,913.701670,362.468513,1203.616407,34013.125881,18035.586407,51976.158513
1,051Y,2019,844.767460,499.676934,1271.890618,696.884828,592.011670,1216.342723,710.846934,194.087460,...,1014.553249,839.676407,452.003249,1219.125881,895.160092,361.075355,1183.681670,34123.683776,16897.049039,50948.179039
2,069P,2019,1028.359565,777.922197,1733.727986,1057.104828,1234.971144,2219.522197,1677.549565,829.861144,...,1701.873776,1005.616934,949.870092,1882.933249,1454.265881,844.720092,2226.432197,54044.636407,38624.642197,92596.724828
3,09NT,2019,874.057460,311.689565,1113.193249,778.563776,311.548513,1017.558513,958.032197,450.646934,...,1068.294302,857.139039,460.433249,1245.018513,1292.113249,251.096934,1470.656407,42267.637986,14241.349039,56436.433249
4,0A91,2019,435.966407,623.383776,986.796407,555.606407,764.137460,1247.190092,681.057986,763.935355,...,888.752723,363.507460,750.272197,1041.225881,617.942723,402.145355,947.534302,21632.906934,24928.092197,46488.445355


In [4]:
# Drop all total & ytd columns
df = df[df.columns.drop(list(df.filter(regex='ttl')))]
df = df.drop(columns=['ytd_youtube', 'ytd_tiktok'])

# Convert week_org columns into rows 
df_sales = df.melt(id_vars=['id', 'fscl_yr_num'],
                  var_name='w',
                  value_name='sales')
df_sales.head()

,id,fscl_yr_num,w,sales
0,002G,2019,wk1_youtube,719.154302
1,051Y,2019,wk1_youtube,844.767460
2,069P,2019,wk1_youtube,1028.359565
3,09NT,2019,wk1_youtube,874.057460
4,0A91,2019,wk1_youtube,435.966407


In [5]:
# Check nulls
df_sales.isnull().sum()

id                 0
fscl_yr_num        0
w                  0
sales          51840
dtype: int64

In [6]:
# Drop and check nulls
df_sales = df_sales[pd.notnull(df_sales['sales'])]
df_sales.isnull().sum()

id             0
fscl_yr_num    0
w              0
sales          0
dtype: int64

In [7]:
# Parse column w
df_week = df_sales['w'].str.extract('(\d+)')
df_sales['org'] = df_sales['w'].str.split('_').str[1]
df_sales['week'] = df_week.copy()

# Rename column
df_sales.rename(columns={'fscl_yr_num':'year'}, inplace=True)

# Get columns
df_sales = df_sales[['id','org','year','week','sales']]
df_sales

,id,org,year,week,sales
0,002G,youtube,2019,1,719.154302
1,051Y,youtube,2019,1,844.767460
2,069P,youtube,2019,1,1028.359565
3,09NT,youtube,2019,1,874.057460
4,0A91,youtube,2019,1,435.966407
...,...,...,...,...,...
223347,ZWTM,tiktok,2019,52,1172.762723
223348,ZY9N,tiktok,2019,52,573.294302
223349,ZYA0,tiktok,2019,52,96.203776
223350,ZZTQ,tiktok,2019,52,494.458513


In [8]:
# Check types
df_sales.dtypes

id        object
org       object
year       int64
week      object
sales    float64
dtype: object

In [9]:
# Convert week into int to create condition and drop columns later
df_sales['week'] = df_sales['week'].astype('int64')
df_sales.dtypes

id        object
org       object
year       int64
week       int64
sales    float64
dtype: object

In [10]:
# In the weekly_2020 file, the dollar amount is $0 from week 26 
df_future = df_sales[(df_sales['year']==2020) &(df_sales['week']>=26)]
df_future['week'].value_counts()

27    2160
26    2160
28    2160
Name: week, dtype: int64

In [11]:
# Drop all values from 2020 week 26
# Not dropping all values that is $0 because it may have other reasons why it's $0 
df_sales = df_sales.drop(df_sales[(df_sales['year']==2020) &(df_sales['week']>=26)].index)

In [12]:
# Create a function to output date based on week
# First week of the year is week 0
# 1 is Monday
def year_week_to_date(year_week):
    return datetime.strptime(year_week + ' 1', '%Y %W %w')

# Structure the string format to use the function
df_sales['year_week'] = df_sales['year'].astype('str') + ' ' + df_sales['week'].astype('str')

# Create a new date column
df_sales['date'] = df_sales['year_week'].apply(year_week_to_date)

# Drop column 'year_week', 'year', 'week'
df_sales.drop(columns=['year', 'week', 'year_week'], inplace=True)
df_sales

,id,org,sales,date
0,002G,youtube,719.154302,2019-01-07
1,051Y,youtube,844.767460,2019-01-07
2,069P,youtube,1028.359565,2019-01-07
3,09NT,youtube,874.057460,2019-01-07
4,0A91,youtube,435.966407,2019-01-07
...,...,...,...,...
223347,ZWTM,tiktok,1172.762723,2019-12-30
223348,ZY9N,tiktok,573.294302,2019-12-30
223349,ZYA0,tiktok,96.203776,2019-12-30
223350,ZZTQ,tiktok,494.458513,2019-12-30


In [13]:
# Pivot the columns
df_sales_pivot = df_sales.pivot_table(values='sales', columns='org', index=['date', 'id'])

# Remove axis name
df_sales_pivot = df_sales_pivot.rename_axis(None, axis=1) 

# Reset index
df_sales_pivot.reset_index(inplace=True)
df_sales_pivot.head()

,date,id,tiktok,youtube
0,2019-01-07,002G,521.411670,719.154302
1,2019-01-07,051Y,499.676934,844.767460
2,2019-01-07,069P,777.922197,1028.359565
3,2019-01-07,09NT,311.689565,874.057460
4,2019-01-07,0A91,623.383776,435.966407


## Clean store data

In [14]:
# Read store data
df_stores = pd.read_excel(Path("data/stores.xlsx"))
df_stores.head()

,id,city,county,state,zip_code
0,MV8N,HUNTSVILLE,MADISON,AL,35802.0
1,MYAA,LANETT,CHAMBERS,AL,36863.0
2,9ZS7,HUNTSVILLE,MADISON,AL,NaN
3,A0HM,HUNTSVILLE,MADISON,AL,35810.0
4,A21M,AUBURN,LEE,AL,36830.0


In [15]:
df_stores.dtypes

id           object
city         object
county       object
state        object
zip_code    float64
dtype: object

In [16]:
# Check for Nulls
df_stores.isnull().sum()

id           0
city         0
county       0
state        0
zip_code    22
dtype: int64

In [17]:
df_stores.sort_values(['state', 'county', 'city'], inplace=True)
df_stores.reset_index(inplace=True)
df_stores.drop(columns='index', inplace=True)

In [18]:
# Take a look of places that have null zip code
df_stores[df_stores['zip_code'].isnull()]

,id,city,county,state,zip_code
4,9ZS7,HUNTSVILLE,MADISON,AL,NaN
77,6IMP,ATLANTA,DEKALB,GA,NaN
167,PV5N,AUGUSTA,RICHMOND,GA,NaN
251,6G1W,CRAWFORDSVILLE,MONTGOMERY,IN,NaN
358,M9YA,LAKE CHARLES,CALCASIEU,LA,NaN
398,6HOF,WARREN,MACOMB,MI,NaN
430,6HGC,WALLED LAKE,OAKLAND,MI,NaN
431,ARF9,WALLED LAKE,OAKLAND,MI,NaN
438,6GA0,ANN ARBOR,WASHTENAW,MI,NaN
507,6HM9,SPRINGFIELD,CLARK,OH,NaN


In [19]:
# Replace null with next zip if state, county, and city are the same
df_stores['zip_code'] = df_stores.groupby(['state', 'county', 'city'])['zip_code'].fillna(method='bfill')

# Replace null with previous zip if state, county, and city are the same
df_stores['zip_code'] = df_stores.groupby(['state', 'county', 'city'])['zip_code'].fillna(method='ffill')

# 5 nulls left
df_stores.isnull().sum()

id          0
city        0
county      0
state       0
zip_code    5
dtype: int64

In [20]:
def to_zip_code(city, state):   
    '''Return zip codes based on city and state'''
    
    return float(search_geo.by_city_and_state(city, state)[0].zipcode)

In [21]:
df_stores['zip_code'] = df_stores.apply(lambda x: to_zip_code(x['city'], x['state']) if pd.isnull(x['zip_code']) else x['zip_code'], axis=1)

In [22]:
df_stores.isnull().sum()

id          0
city        0
county      0
state       0
zip_code    0
dtype: int64

In [23]:
df_stores['zip_code'] = df_stores['zip_code'].astype('int').astype('str')

In [24]:
df_stores.dtypes

id          object
city        object
county      object
state       object
zip_code    object
dtype: object

In [25]:
df_stores.head()

,id,city,county,state,zip_code
0,MYAA,LANETT,CHAMBERS,AL,36863
1,A21M,AUBURN,LEE,AL,36830
2,CR62,OPELIKA,LEE,AL,36801
3,MV8N,HUNTSVILLE,MADISON,AL,35802
4,9ZS7,HUNTSVILLE,MADISON,AL,35810


In [26]:
# Merge weekly sales and location info based on id
df_store_sale = df_sales_pivot.merge(df_stores, on='id', how='inner')
df_store_sale.head()

,date,id,tiktok,youtube,city,county,state,zip_code
0,2019-01-07,002G,521.411670,719.154302,SMYRNA,COBB,GA,30080
1,2019-01-14,002G,705.117986,700.197460,SMYRNA,COBB,GA,30080
2,2019-01-21,002G,175.349565,773.804302,SMYRNA,COBB,GA,30080
3,2019-01-28,002G,409.928513,903.122197,SMYRNA,COBB,GA,30080
4,2019-02-04,002G,378.533249,829.431144,SMYRNA,COBB,GA,30080


In [28]:
# Output data to csv
df_store_sale.to_csv(Path("data/stores_revenue.csv"),index=False)